In [1]:
import yfinance
from finta import TA
import yfinance_utils
import pandas as pd

TICK = "goog"
ohlc = pd.read_csv(f"../datasets/{TICK.upper()}")
print(ohlc)

/Users/xtra/Documents/GitHub/yfinance_utils/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


           Date   Adj Close       Close        High         Low        Open  \
0    2023-12-19  137.605576  138.100006  138.770004  137.449997  138.000000   
1    2023-12-20  139.159988  139.660004  143.078003  139.410004  140.330002   
2    2023-12-21  141.292328  141.800003  142.029999  140.473007  140.770004   
3    2023-12-22  142.209030  142.720001  143.250000  142.054993  142.130005   
4    2023-12-26  142.308670  142.820007  143.945007  142.500000  142.979996   
..          ...         ...         ...         ...         ...         ...   
246  2024-12-11  196.710007  196.710007  196.889999  186.259995  186.699997   
247  2024-12-12  193.630005  193.630005  196.705002  193.279999  196.300003   
248  2024-12-13  191.380005  191.380005  194.339996  191.259995  192.710007   
249  2024-12-16  198.160004  198.160004  200.639999  194.110001  194.365005   
250  2024-12-17  197.119995  197.119995  202.880005  196.690002  198.529999   

       Volume  
0    20661000  
1    33507300  
2  

In [2]:
mfi = TA.MFI(ohlc)
print(mfi)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
246    76.612746
247    88.844405
248    89.649287
249    90.187042
250    90.586759
Name: 14 period MFI, Length: 251, dtype: float64


In [3]:
print(TA.RSI(ohlc))

0             NaN
1      100.000000
2      100.000000
3      100.000000
4      100.000000
          ...    
246     75.195533
247     69.967019
248     66.338194
249     71.187556
250     69.532819
Name: 14 period RSI, Length: 251, dtype: float64


In [4]:
print(TA.BBANDS(ohlc))

       BB_UPPER   BB_MIDDLE    BB_LOWER
0           NaN         NaN         NaN
1           NaN         NaN         NaN
2           NaN         NaN         NaN
3           NaN         NaN         NaN
4           NaN         NaN         NaN
..          ...         ...         ...
246  189.300824  175.792002  162.283180
247  192.037861  176.449002  160.860143
248  194.112319  177.150502  160.188685
249  197.656253  178.364002  159.071751
250  200.389289  179.380002  158.370715

[251 rows x 3 columns]


In [5]:
print(TA.KC(ohlc))

       KC_UPPER    KC_LOWER
0           NaN         NaN
1           NaN         NaN
2           NaN         NaN
3           NaN         NaN
4           NaN         NaN
..          ...         ...
246  185.886795  168.447189
247  187.846702  169.623093
248  189.242999  170.635388
249  192.164313  171.184705
250  194.511310  171.779706

[251 rows x 2 columns]


In [6]:
print(TA.CHANDELIER(ohlc))

         Short.       Long.
0           NaN         NaN
1           NaN         NaN
2           NaN         NaN
3           NaN         NaN
4           NaN         NaN
..          ...         ...
246  178.258141  183.941856
247  178.422459  183.777538
248  178.428597  183.771400
249  179.254958  186.695039
250  179.490867  188.699136

[251 rows x 2 columns]


In [7]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ohlc['diffup'] = (ohlc["High"] - ohlc['Close']) / ohlc['Close'] * 10000
ohlc['diffdown'] = abs((ohlc["Close"] - ohlc['Low']) / ohlc['Close'] * 10000)

fig = px.line(ohlc, x="Date", y=["High", "Close"], title="Volume")
fig.update_layout(title=TICK, title_x=0.5)

fig.show()


In [8]:
fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.update_layout(title=TICK, title_x=0.5)

fig2.add_trace(
    go.Scatter(x=ohlc['Date'], y=ohlc['Close'], name="Close", marker_color='rgb(100, 100, 255)'),
    secondary_y=False,
)
fig2.add_trace(
    go.Scatter(x=ohlc['Date'], y=ohlc['High'], name="High", marker_color='rgb(0, 0, 255)'),
    secondary_y=False,
)
fig2.add_trace(
    go.Scatter(x=ohlc['Date'], y=ohlc['Low'], name="Low", marker_color='rgb(0, 255, 255)'),
    secondary_y=False,
)
fig2.add_trace(
    go.Scatter(x=ohlc['Date'], y=ohlc['diffup'], name="diff up", marker_color='rgb(200, 100, 200)'),
    secondary_y=True,
)
fig2.add_trace(
    go.Scatter(x=ohlc['Date'], y=ohlc['diffdown'], name="diff down", marker_color='rgb(255, 200, 200)'),
    secondary_y=True,
)
# ohlc['pct'] = (ohlc['diffup'] + ohlc['diffdown']) / 2

# fig2.add_trace(
#     go.Scatter(x=ohlc['Date'], y=ohlc['pct'], name="pct", marker_color='rgb(255, 200, 200)'),
#     secondary_y=True,
# )

fig2.show()

In [9]:


# Create figure with secondary y-axis

fig1 = make_subplots(specs=[[{"secondary_y": True}]])
fig1.update_layout(title=TICK, title_x=0.5)
vol = ohlc['Volume'].rolling(14).mean()

from sklearn.preprocessing import minmax_scale
minmaxvol = minmax_scale(vol, feature_range=(1,100))
minmaxclose = minmax_scale(ohlc['Close'].rolling(14).mean(), feature_range=(1,100))

# Add traces
fig1.add_trace(
    go.Scatter(x=ohlc['Date'], y=minmaxvol, name="Volume", marker_color='rgb(0, 0, 255)'),
    secondary_y=True,
)
# fig1.add_trace(
#     go.Scatter(x=ohlc['Date'], y=ohlc['Volume'], name="Volume", marker_color='rgb(0, 0, 255)'),
#     secondary_y=False,
# )
fig1.add_trace(
    go.Scatter(x=ohlc['Date'], y=minmaxclose, name="Close", marker_color='rgb(0, 255, 0)'),
    secondary_y=False,
)
fig1.add_trace(
    go.Scatter(x=ohlc['Date'], y=TA.RSI(ohlc), name="RSI", marker_color='rgb(255, 0, 0)'),
    secondary_y=True,
)
# fig1.add_trace(
#     go.Scatter(x=ohlc['Date'], y=TA.MFI(ohlc), name="MFI", marker_color='rgb(255, 200, 0)'),
#     secondary_y=True,
# )

fig1.show()